# EDA

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

: 

In [ ]:
# Set up plotting style
plt.style.use('seaborn')
sns.set_style("whitegrid")

In [ ]:
ct_path = "3702_left_knee.nii.gz"
mask_path = "original_mask.nii.gz"